In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import time
import unittest
import tqdm.notebook as tq
import solitaire_env
import sys
import seaborn as sns
import pickle as pkl
import gp
sys.setrecursionlimit(10**6)

In [4]:
with open("games/run_2020-05-03 07:17:14.501735.pkl","rb") as file:
    run_data = pkl.load(file)

In [5]:
actions_matrix = run_data['actions_matrix']

In [6]:
iterations_used_per_game = run_data['iterations_used_per_game']
it = pd.DataFrame(data = {"iterations":iterations_used_per_game})['iterations'].value_counts()

In [7]:
max_action_take = max(it.index)
max_action_take

96

In [8]:
for game_no,action_seq,end_state,start_state,won in actions_matrix:
    if len(action_seq) == max_action_take:
        break;

In [9]:
game_no,won,len(action_seq)

(847, False, 96)

In [10]:
"".join([str(x) for x in action_seq])

'111010222001111222203333323211111111132220111111111111134441142044441411134344144414144111444411'

In [11]:
start_env = solitaire_env.env()
start_env.state = start_env.generate_deep_copy_state(start_state)

start_env.get_foundation()

printing foundation number 1 ********************
printing foundation number 2 ********************
printing foundation number 3 ********************
printing foundation number 4 ********************


In [12]:
start_env.get_pile()

suit = diamond    color = red        number =    10 speciality = None       face = up   
suit = club       color = black      number =     6 speciality = None       face = up   
suit = spade      color = black      number =     7 speciality = None       face = up   
suit = club       color = black      number =     7 speciality = None       face = up   
suit = heart      color = red        number =     7 speciality = None       face = up   
suit = club       color = black      number =     1 speciality = ACE        face = up   
suit = diamond    color = red        number =     4 speciality = None       face = up   
suit = diamond    color = red        number =    11 speciality = JACK       face = up   
suit = diamond    color = red        number =     6 speciality = None       face = up   
suit = heart      color = red        number =     8 speciality = None       face = up   
suit = club       color = black      number =    10 speciality = None       face = up   
suit = spade      col

In [13]:
start_env.get_tableau()

printing tableau number 1 ********************
suit = club       color = black      number =    13 speciality = KING       face = up   
printing tableau number 2 ********************
suit = heart      color = red        number =     6 speciality = None       face = down 
suit = spade      color = black      number =     6 speciality = None       face = up   
printing tableau number 3 ********************
suit = diamond    color = red        number =     2 speciality = None       face = down 
suit = spade      color = black      number =    10 speciality = None       face = down 
suit = spade      color = black      number =     4 speciality = None       face = up   
printing tableau number 4 ********************
suit = heart      color = red        number =     5 speciality = None       face = down 
suit = spade      color = black      number =    13 speciality = KING       face = down 
suit = club       color = black      number =    12 speciality = QUEEN      face = down 
suit = diam

In [52]:
default_action_matrix = [0]
default_action_freq = {0:0,1:0,2:0,3:0,4:0,5:0}

In [53]:
steps,won = greedy_policy.greedy_policy(start_env,20000,default_action_freq,default_action_matrix,0,fp_flag=True)

In [54]:
steps,won

(124, False)

In [102]:

takes = []
for a in action_seq:
    action = int(a)
    w,t = start_env.step(action,fp_flag=True)
    takes.append(int(t))

In [103]:
print(takes),sum(takes)

[1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


(None, 29)

In [104]:
sum(takes)

29

In [105]:
start_env.get_foundation()

printing foundation number 1 ********************
suit = club       color = black      number =     1 speciality = ACE        face = up   
suit = club       color = black      number =     2 speciality = None       face = up   
printing foundation number 2 ********************
suit = heart      color = red        number =     1 speciality = ACE        face = up   
suit = heart      color = red        number =     2 speciality = None       face = up   
printing foundation number 3 ********************
suit = diamond    color = red        number =     1 speciality = ACE        face = up   
suit = diamond    color = red        number =     2 speciality = None       face = up   
suit = diamond    color = red        number =     3 speciality = None       face = up   
suit = diamond    color = red        number =     4 speciality = None       face = up   
suit = diamond    color = red        number =     5 speciality = None       face = up   
suit = diamond    color = red        number =    

In [106]:
start_env.get_pile()

suit = diamond    color = red        number =    13 speciality = KING       face = up   
suit = heart      color = red        number =     9 speciality = None       face = up   
suit = spade      color = black      number =    11 speciality = JACK       face = up   
suit = heart      color = red        number =    12 speciality = QUEEN      face = up   
suit = club       color = black      number =    13 speciality = KING       face = up   
suit = diamond    color = red        number =    10 speciality = None       face = up   
suit = spade      color = black      number =     9 speciality = None       face = up   
suit = spade      color = black      number =     4 speciality = None       face = up   


In [107]:
start_env.get_tableau()

printing tableau number 1 ********************
suit = club       color = black      number =     5 speciality = None       face = up   
printing tableau number 2 ********************
suit = club       color = black      number =     7 speciality = None       face = down 
suit = heart      color = red        number =    13 speciality = KING       face = up   
suit = club       color = black      number =    12 speciality = QUEEN      face = up   
suit = heart      color = red        number =    11 speciality = JACK       face = up   
suit = club       color = black      number =    10 speciality = None       face = up   
suit = diamond    color = red        number =     9 speciality = None       face = up   
suit = spade      color = black      number =     8 speciality = None       face = up   
suit = heart      color = red        number =     7 speciality = None       face = up   
suit = spade      color = black      number =     6 speciality = None       face = up   
printing tableau

In [14]:
end_en = solitaire_env.env()
end_en.state = end_state
end_en.state = end_en.copy_state()


In [19]:
end_en.get_foundation()

printing foundation number 1 ********************
suit = club       color = black      number =     1 speciality = ACE        face = up   
suit = club       color = black      number =     2 speciality = None       face = up   
suit = club       color = black      number =     3 speciality = None       face = up   
suit = club       color = black      number =     4 speciality = None       face = up   
printing foundation number 2 ********************
suit = heart      color = red        number =     1 speciality = ACE        face = up   
suit = heart      color = red        number =     2 speciality = None       face = up   
suit = heart      color = red        number =     3 speciality = None       face = up   
printing foundation number 3 ********************
suit = diamond    color = red        number =     1 speciality = ACE        face = up   
suit = diamond    color = red        number =     2 speciality = None       face = up   
suit = diamond    color = red        number =    

In [20]:
end_en.get_tableau()

printing tableau number 1 ********************
suit = club       color = black      number =    13 speciality = KING       face = up   
suit = diamond    color = red        number =    12 speciality = QUEEN      face = up   
suit = club       color = black      number =    11 speciality = JACK       face = up   
suit = heart      color = red        number =    10 speciality = None       face = up   
suit = club       color = black      number =     9 speciality = None       face = up   
suit = heart      color = red        number =     8 speciality = None       face = up   
suit = club       color = black      number =     7 speciality = None       face = up   
printing tableau number 2 ********************
suit = heart      color = red        number =     6 speciality = None       face = up   
suit = club       color = black      number =     5 speciality = None       face = up   
suit = heart      color = red        number =     4 speciality = None       face = up   
printing tableau

In [21]:
end_en.get_pile()

In [22]:
w,t = end_en.step(1,fp_flag=True)
w,t

(False, True)

In [24]:
end_en.get_foundation()

printing foundation number 1 ********************
suit = club       color = black      number =     1 speciality = ACE        face = up   
suit = club       color = black      number =     2 speciality = None       face = up   
suit = club       color = black      number =     3 speciality = None       face = up   
suit = club       color = black      number =     4 speciality = None       face = up   
printing foundation number 2 ********************
suit = heart      color = red        number =     1 speciality = ACE        face = up   
suit = heart      color = red        number =     2 speciality = None       face = up   
suit = heart      color = red        number =     3 speciality = None       face = up   
suit = heart      color = red        number =     4 speciality = None       face = up   
printing foundation number 3 ********************
suit = diamond    color = red        number =     1 speciality = ACE        face = up   
suit = diamond    color = red        number =    

In [25]:
end_en.get_tableau()

printing tableau number 1 ********************
suit = club       color = black      number =    13 speciality = KING       face = up   
suit = diamond    color = red        number =    12 speciality = QUEEN      face = up   
suit = club       color = black      number =    11 speciality = JACK       face = up   
suit = heart      color = red        number =    10 speciality = None       face = up   
suit = club       color = black      number =     9 speciality = None       face = up   
suit = heart      color = red        number =     8 speciality = None       face = up   
suit = club       color = black      number =     7 speciality = None       face = up   
printing tableau number 2 ********************
suit = heart      color = red        number =     6 speciality = None       face = up   
suit = club       color = black      number =     5 speciality = None       face = up   
printing tableau number 3 ********************
suit = heart      color = red        number =    13 specia

In [26]:
w,t = end_en.step(4,fp_flag=True)
w,t

(False, True)

In [27]:
end_en.get_foundation()

printing foundation number 1 ********************
suit = club       color = black      number =     1 speciality = ACE        face = up   
suit = club       color = black      number =     2 speciality = None       face = up   
suit = club       color = black      number =     3 speciality = None       face = up   
suit = club       color = black      number =     4 speciality = None       face = up   
printing foundation number 2 ********************
suit = heart      color = red        number =     1 speciality = ACE        face = up   
suit = heart      color = red        number =     2 speciality = None       face = up   
suit = heart      color = red        number =     3 speciality = None       face = up   
printing foundation number 3 ********************
suit = diamond    color = red        number =     1 speciality = ACE        face = up   
suit = diamond    color = red        number =     2 speciality = None       face = up   
suit = diamond    color = red        number =    

In [28]:
end_en.get_tableau()

printing tableau number 1 ********************
suit = club       color = black      number =    13 speciality = KING       face = up   
suit = diamond    color = red        number =    12 speciality = QUEEN      face = up   
suit = club       color = black      number =    11 speciality = JACK       face = up   
suit = heart      color = red        number =    10 speciality = None       face = up   
suit = club       color = black      number =     9 speciality = None       face = up   
suit = heart      color = red        number =     8 speciality = None       face = up   
suit = club       color = black      number =     7 speciality = None       face = up   
printing tableau number 2 ********************
suit = heart      color = red        number =     6 speciality = None       face = up   
suit = club       color = black      number =     5 speciality = None       face = up   
suit = heart      color = red        number =     4 speciality = None       face = up   
printing tableau

In [31]:
w,t = end_en.step(1,fp_flag=True)
w,t

(False, False)

In [30]:
end_en.get_foundation()

printing foundation number 1 ********************
suit = club       color = black      number =     1 speciality = ACE        face = up   
suit = club       color = black      number =     2 speciality = None       face = up   
suit = club       color = black      number =     3 speciality = None       face = up   
suit = club       color = black      number =     4 speciality = None       face = up   
printing foundation number 2 ********************
suit = heart      color = red        number =     1 speciality = ACE        face = up   
suit = heart      color = red        number =     2 speciality = None       face = up   
suit = heart      color = red        number =     3 speciality = None       face = up   
printing foundation number 3 ********************
suit = diamond    color = red        number =     1 speciality = ACE        face = up   
suit = diamond    color = red        number =     2 speciality = None       face = up   
suit = diamond    color = red        number =    

In [103]:
end_en.get_tableau()

printing tableau number 1 ********************
printing tableau number 2 ********************
suit = club       color = black      number =    13 speciality = KING       face = up   
printing tableau number 3 ********************
suit = club       color = black      number =    12 speciality = QUEEN      face = up   
suit = heart      color = red        number =    11 speciality = JACK       face = up   
suit = spade      color = black      number =    10 speciality = None       face = up   
suit = diamond    color = red        number =     9 speciality = None       face = up   
suit = spade      color = black      number =     8 speciality = None       face = up   
suit = heart      color = red        number =     7 speciality = None       face = up   
suit = spade      color = black      number =     6 speciality = None       face = up   
suit = heart      color = red        number =     5 speciality = None       face = up   
printing tableau number 4 ********************
suit = hear